## Hugging_Face Embeddings

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

/home/hammadali08/PycharmProjects/RAG/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [4]:
vectors=embeddings.embed_query("Hello I am Hammad Ali Tahir")
print(vectors)
print(len(vectors))

[0.008959689177572727, 0.09178251028060913, -0.0042881350964307785, 0.05720607936382294, -0.07849406450986862, -0.05122251436114311, 0.16765734553337097, -0.05901440978050232, 0.07593921571969986, -0.010059552267193794, 0.08655273169279099, -0.09947802871465683, 0.021933089941740036, -0.06579573452472687, 0.07003957033157349, -0.05746196582913399, 0.020773770287632942, -0.02633540891110897, -0.03527873381972313, -0.0733388140797615, -0.11658710241317749, -0.03513731807470322, 0.006203813012689352, 0.032315172255039215, -0.11720556020736694, -0.025680026039481163, 0.006114559713751078, 0.09403189271688461, 0.05834360793232918, -0.06230120733380318, -0.017115095630288124, 0.03678779676556587, 0.07167858630418777, -0.012926035560667515, -0.012077664956450462, 0.12244991958141327, -0.01254356186836958, -0.016596350818872452, 0.014732501469552517, 0.030170902609825134, 0.028035953640937805, -0.041900791227817535, 0.006529353559017181, -0.06054788827896118, 0.08014234900474548, -0.0735695436

### Applying over real world pdf

In [82]:
from typing import List, Dict, Optional
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

class SmartPDFProcessor:
    """Advanced PDF processing with error handling + custom metadata injection"""

    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 10):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=[" "],
        )

    def process_pdf(
        self, 
        pdf_path: str, 
        custom_metadata: Optional[Dict[str, str]] = None
    ) -> List[Document]:
        """Process PDF with smart chunking and metadata enhancement"""

        # Load PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        processed_chunks = []

        for page_num, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)

            # Skip nearly empty pages
            if len(cleaned_text.strip()) < 50:
                continue

            # Step 1: Page details FIRST
            metadata = {
                "page": page_num + 1,
                "total_pages": len(pages),
                "chunk_method": "smart_pdf_processor",
                "char_count": len(cleaned_text),
            }

            # Step 2: Add PDF metadata (if any)
            if page.metadata:
                metadata.update(page.metadata)

            # Step 3: Add custom metadata (overrides everything if same key)
            if custom_metadata:
                metadata.update(custom_metadata)

            # Create chunks
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[metadata]
            )

            processed_chunks.extend(chunks)

        return processed_chunks

    def _clean_text(self, text: str) -> str:
    # Remove excessive whitespace
        text = " ".join(text.split())
    
    # Fix common PDF extraction issues
        text = text.replace("ﬁ", "fi")
        text = text.replace("ﬂ", "fl")
    
        return text


In [131]:
processed_data=SmartPDFProcessor()
processed_chunks=processed_data.process_pdf("/home/hammadali08/Personal/FYP Datasets/A Brief History of Pakistan By JAMES WYNBRANDT.pdf",custom_metadata={"source":"project.pdf"})

In [109]:
Book_embeddings=embeddings.embed_documents([doc.page_content for doc in processed_chunks])

In [125]:
processed_chunks[1458]

Document(metadata={'page': 289, 'total_pages': 337, 'chunk_method': 'smart_pdf_processor', 'char_count': 2703, 'producer': 'Acrobat Distiller 7.0.5 for Macintosh', 'creator': 'Adobe InDesign CS2 (4.0.5)', 'creationdate': '2008-10-28T11:49:49-04:00', 'title': 'Brief History of Pakistan', 'author': 'Wynbrandt, James.', 'moddate': '2009-07-28T15:09:17-05:00', 'ebx_publisher': 'Facts on File, Inc.', 'source': 'project.pdf', 'page_label': '277'}, page_content='to change the constitution and weaken Musharraf’s powers as president, the two parties began discussions on forming a coalition government. The major issue dividing the two parties involved the reinstatement of the 60 dismissed judges, whose removal by Musharraf had been one of the lightning rods for opposition to his regime. Sharif and the PML sought the judges’ immediate reinstatement. The PPP preferred to avoid a confrontation with Musharraf they felt an immediate reinstate- ment would')

In [126]:
print(len(Book_embeddings))


1706


### Cosine Similarity:

In [111]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)

### Semantic Similiarity

In [113]:
def semantic_search(query: str, embeddings_list: List[List[float]], top_k: int = 5) -> List[int]:
    """Perform semantic search to find top_k similar documents."""
    query_embedding = embeddings.embed_query(query)
    similarities = [cosine_similarity(query_embedding, doc_embedding) for doc_embedding in embeddings_list]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return top_indices.tolist()

In [127]:
data=semantic_search("Punjab",Book_embeddings,top_k=4)

In [128]:
data

[1074, 0, 64, 1574]

In [129]:
print(processed_chunks[1074])

page_content='that had been adopted in part to mollify East Pakistan, the nation was to have four provinces: Punjab, Sind, NWFP , and 001-311_BH-Pakistan_ch.indd 206 10/28/08 11:45:26 AM' metadata={'page': 218, 'total_pages': 337, 'chunk_method': 'smart_pdf_processor', 'char_count': 2607, 'producer': 'Acrobat Distiller 7.0.5 for Macintosh', 'creator': 'Adobe InDesign CS2 (4.0.5)', 'creationdate': '2008-10-28T11:49:49-04:00', 'title': 'Brief History of Pakistan', 'author': 'Wynbrandt, James.', 'moddate': '2009-07-28T15:09:17-05:00', 'ebx_publisher': 'Facts on File, Inc.', 'source': 'project.pdf', 'page_label': '206'}


In [130]:
print(Book_embeddings[1074])

[0.02990679442882538, 0.07751582562923431, -0.014368574135005474, 0.010206487029790878, -0.017249001190066338, -0.05095590278506279, -0.018334854394197464, -0.06946925073862076, -0.025440474972128868, -0.04104847088456154, 0.06818060576915741, -0.03668774664402008, 0.13188011944293976, 0.05383097380399704, -0.004409657325595617, -0.06285691261291504, -0.06299027055501938, 0.006179946940392256, 0.0077254557982087135, -0.07811947911977768, -0.0027256205212324858, -0.035614024847745895, 0.09553293138742447, -0.06223505362868309, -0.024342380464076996, -0.01982978731393814, 0.02601710520684719, -0.011741602793335915, 0.01595187373459339, 0.03426104038953781, 0.01263832114636898, 0.07665934413671494, 0.024127142503857613, 0.038648251444101334, 0.051454365253448486, 0.006660631857812405, 0.010166623629629612, 0.05563095957040787, -0.01764943078160286, 0.05138678476214409, 0.0258815698325634, 0.013013393618166447, 0.08860703557729721, 0.07946359366178513, -0.020656457170844078, 0.013354632072